In [113]:
import pandas as pd
from sqlalchemy import false
from tqdm import *

In [158]:
data_athletes = pd.read_csv('data/athletes.csv')
data_programs = pd.read_csv('data/programs.csv')
data_medal_counts = pd.read_csv('data/medal_counts.csv')
team_df = pd.read_csv('data/team_index.csv')
years_index = range(1896, 2025, 4)
programs_index = pd.read_csv('data/programs_index.csv')['program_id'].tolist()

,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,Artistic Swimming,SWA,World Aquatics,0,0,0,0,0,0,...,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,Aquatics,Diving,DIV,World Aquatics,0,0,2,1,2,4,...,4.0,4.0,4.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,Aquatics,Marathon Swimming,OWS,World Aquatics,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
3,Aquatics,Swimming,SWM,World Aquatics,4,7,9,4,6,9,...,31.0,31.0,32.0,32.0,32.0,32.0,32.0,32.0,35.0,35.0
4,Aquatics,Water Polo,WPO,World Aquatics,0,1,1,0,1,1,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Skating,Figure,FSK,ISU,0,0,0,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Ice Hockey,Ice Hockey,IHO,IIHF,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Total events,Total events,Total events,Total events,43,97,95,76,110,102,...,237.0,257.0,271.0,300.0,301.0,302.0,302.0,306.0,339.0,329.0
72,Total disciplines,Total disciplines,Total disciplines,Total disciplines,10,22,18,13,25,18,...,31.0,34.0,37.0,40.0,40.0,42.0,40.0,42.0,50.0,48.0


In [195]:
number_of_years_considered = 3
number_of_years_start = 2
medal_score = {
    "Gold": 3,
    "Silver": 2,
    "Bronze": 1
}
# team_not_include_in_athletes = []
# team_not_match_in_any_program = []
# team_not_save = []

In [196]:
def make_dataset(noc, team_now, team_sum, predict_data_2028=False):
    tmp_athletes = data_athletes[data_athletes["NOC"] == noc]
    result_df = pd.DataFrame()
    if tmp_athletes.shape[0] == 0:
        team_not_include_in_athletes.append(noc)
        # print("No such team: " + noc)
        return result_df
        # raise ValueError("No such team: " + team_name + " (" + noc + ")")
    num_participate = 0

    # each year
    if not predict_data_2028:
        tqdm_bar = tqdm(years_index)
    else:
        tqdm_bar = [2024]
    for year in tqdm_bar:
        tmp_one_year = tmp_athletes[tmp_athletes["Year"] == year]

        if tmp_one_year.shape[0] == 0:
            continue
        elif tmp_one_year.shape[0] != 0 and num_participate <= number_of_years_start and not predict_data_2028:
            num_participate += 1
            continue

        if predict_data_2028:
            if noc == "USA":
                host = 1
            else:
                host = 0
        else:
            host = tmp_one_year["Host"].iloc[0]
        tmp_one_year_medal = data_medal_counts[data_medal_counts["Year"] == year]
        sum_g = tmp_one_year_medal["Gold"].sum()
        sum_s = tmp_one_year_medal["Silver"].sum()
        sum_b = tmp_one_year_medal["Bronze"].sum()
        tmp_one_year_medal = tmp_one_year_medal[tmp_one_year_medal["NOC"] == noc]

        if tmp_one_year_medal.shape[0] != 0:
            rate_g = tmp_one_year_medal["Gold"].sum() / sum_g
            rate_s = tmp_one_year_medal["Silver"].sum() / sum_s
            rate_b = tmp_one_year_medal["Bronze"].sum() / sum_b
        else:
            rate_g = 0
            rate_s = 0
            rate_b = 0

        tmp_df = pd.DataFrame()
        tmp_df["Year"] = [year]
        tmp_df["NOC"] = [noc]
        if not predict_data_2028:
            tmp_df["Gold_rate"] = [rate_g]
            tmp_df["Silver_rate"] = [rate_s]
            tmp_df["Bronze_rate"] = [rate_b]
        tmp_df["Host"] = [host]

        # each program
        for program in programs_index:
            tmp_one_year_program = tmp_one_year[tmp_one_year["Sport"] == program]
            if tmp_one_year_program.shape[0] == 0:
                tmp_df[program] = [0]
                continue
            tmp_one_year_program_athletes = list(set(tmp_one_year_program["Name"].tolist()))

            team_score_in_program = 0
            athletes_num_in_program = 0

            for each_athletes in tmp_one_year_program_athletes:
                if predict_data_2028:
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] <= year) & (tmp_athletes["Year"] >= (year - 4 * number_of_years_considered))]
                else:
                    tmp_athletes_record = tmp_athletes[(tmp_athletes["Name"] == each_athletes) & (tmp_athletes["Year"] < year) & (tmp_athletes["Year"] >= (year - 4 * number_of_years_considered))]

                if tmp_athletes_record.shape[0] == 0:
                    athletes_num_in_program += 1
                    continue
                else:
                    number_of_participation = len(list(set(tmp_athletes_record["Year"])))
                    team_score_in_program += tmp_athletes_record["Medal"].sum() / number_of_participation
                    athletes_num_in_program += 1

            # print(year)
            # tmp = data_programs[data_programs["Code"] == program]
            # return tmp["2004"]
            all_program_event = (data_programs[data_programs["Code"] == program])[str(year)].iloc[0]
            tmp_df[program] = [team_score_in_program / all_program_event]

        result_df = pd.concat([result_df, tmp_df], ignore_index=True)

        if result_df.shape[0] == 0:
            team_not_match_in_any_program.append(noc)
            return result_df
        if not predict_data_2028:
            tqdm_bar.set_description('Team [{}/{}], NOC: {}, Year: {}, Usable:{} '.format(team_now, team_sum, noc, year, result_df.shape[0]))

        if predict_data_2028:
            break

    result_df.fillna(0, inplace=True)
    return result_df


In [197]:
data_athletes["Medal"] = data_athletes["Medal"].replace("Gold", medal_score["Gold"])
data_athletes["Medal"] = data_athletes["Medal"].replace("Silver", medal_score["Silver"])
data_athletes["Medal"] = data_athletes["Medal"].replace("Bronze", medal_score["Bronze"])
data_athletes["Medal"] = data_athletes["Medal"].replace("No medal", 0)
team_sum = len(team_df)

team_not_include_in_athletes = []
team_not_save = []
team_not_match_in_any_program = []
# make_dataset("CHN", 1, 206)

# Train dataset from each team

In [198]:
print("Make train dataset for each team")
for i in range(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    df.to_csv("data/dataset/train/" + noc + ".csv", index=False)

Make train dataset for each team


Team [206/206], NOC: LES, Year: 2024, Usable:8 : 100%|██████████| 33/33 [00:00<00:00, 174.69it/s]


# Train dataset from all team

In [199]:
print("Make sum dataset (every country in one dataset)")
sum_dataset_df = pd.DataFrame()
for i in range(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    sum_dataset_df = pd.concat([sum_dataset_df, df], ignore_index=True)
sum_dataset_df.to_csv("data/dataset/sum_dataset.csv", index=False)

Make sum dataset (every country in one dataset)


Team [206/206], NOC: LES, Year: 2024, Usable:8 : 100%|██████████| 33/33 [00:00<00:00, 178.45it/s]


# Predict dataset 2028

In [200]:
print("Make predict dataset for each team")
for i in trange(team_sum):
    noc = team_df.iloc[i]["NOC"]
    df = make_dataset(noc, i+1, team_sum, predict_data_2028=True)
    if df.shape[0] == 0:
        team_not_save.append(noc)
        continue
    df.to_csv("data/dataset/predict/" + noc + ".csv", index=False)

Make predict dataset for each team


100%|██████████| 206/206 [00:14<00:00, 13.96it/s]


In [136]:
f

([], [], [])